# Practica 2

**Objetivo:** A partir del [corpus proporcionado](./../99_corpus/corpusML.txt) realizar un modelo del lenguaje neuronal con base en la arquitectura propuesta por Bengio (2003).

In [1]:
from re import sub
from unicodedata import normalize
from nltk.stem.snowball import SpanishStemmer
from sklearn.model_selection import train_test_split
from itertools import chain
from collections import Counter,defaultdict

#import numpy as np
#import matplotlib.pyplot as plt
#from scipy.optimize import minimize as min

def ejemplos(msg, collection, n_elements):
    print(msg)
    for element in collection[:n_elements]:
        print(element)

## 1. Limpiar los textos y aplicar stemming a las palabras.

In [2]:
# Abrir el documento
document = './../99_corpus/corpusML.txt'
text = open(document,'r',encoding='utf-8').read()

In [17]:
# Limpiamos el documento
text_clean = ""
lines = []

for line in text.split("\n"):
    # Eliminamos caracteres compuestos y pasamos a minusculas
    line = normalize('NFKC', line).lower()
    # Eliminamos extensiones y numeros
    line = sub(r"\\.*|{.*}|\\|\[.*\]|[!-@[-`{-~]", ' ', line)
    # Eliminamos signos de puntuacion
    line = sub(r"[^\w]", " ", line)
    # Eliminamos saltos de linea
    line = " ".join(line.split())
    # Si la linea no esta vacia la añadimos al texto limpio
    if line:
        lines.append(line)
        
text_clean += " ".join(lines)
text_clean = text_clean.split()

ejemplos("### Lineas de ejemplo ###\n", lines, 5)

### Lineas de ejemplo ###

comencé a trabajar y me pegaron me maltrataron con chicote
mis patrones me pegaron porque no me quería apurar porque era flojo
por eso me habían pegado
cuando me pegaban ya entonces me quitaba
pues entonces no quise trabajar


In [4]:
# Aplicamos Stemming a los tokens limpios

# Debido a que el corpus se encuentra en español, utilizaremos el stemmer de NLTK para español.
stemmer = SpanishStemmer()
stems = []
for line in lines:
    stems.append([stemmer.stem(word) for word in line.split(" ")])
    
ejemplos("### Lineas de ejemplo ###\n", stems, 5)

### Lineas de ejemplo ###

['comenc', 'a', 'trabaj', 'y', 'me', 'peg', 'me', 'maltrat', 'con', 'chicot']
['mis', 'patron', 'me', 'peg', 'porqu', 'no', 'me', 'quer', 'apur', 'porqu', 'era', 'floj']
['por', 'eso', 'me', 'hab', 'peg']
['cuand', 'me', 'peg', 'ya', 'entonc', 'me', 'quit']
['pues', 'entonc', 'no', 'quis', 'trabaj']


In [5]:
# Separación del corpus en prueba (70%) y evaluación (30%)
train_corpus, eval_corpus = train_test_split(stems, test_size=0.3)

print("### Corpus de entrenamiento ###")
print("\nTamaño del corpus de entrenamiento:",len(train_corpus), " lineas")
ejemplos("\nLineas de ejemplo", train_corpus, 5)

print("\n### Corpus de evaluacion ###")
print("\nTamaño del corpus de evaluación:", len(eval_corpus), " lineas")
ejemplos("\nLineas de ejemplo", eval_corpus, 5)

### Corpus de entrenamiento ###

Tamaño del corpus de entrenamiento: 751  lineas

Lineas de ejemplo
['pues', 'lo', 'pag', 'y', 'estuv', 'bien', 'entonc']
['voy', 'a', 'cuid', 'mi', 'poll']
['mis', 'hoj', 'de', 'maiz', 'todav', 'no', 'me', 'las', 'acab', 'nomas', 'estan']
['dijeron', 'dond', 'ha', 'estad', 'nuestr', 'madr', 'por', 'mas', 'de', 'eso']
['se', 'las', 'llev', 'con', 'burrit', 'nomas']

### Corpus de evaluacion ###

Tamaño del corpus de evaluación: 323  lineas

Lineas de ejemplo
['per', 'el', 'niñ', 'se', 'alegr']
['me', 'iba', 'a', 'andar', 'por', 'alli', 'hast', 'la', 'tard']
['pues', 'si', 'pues', 'no', 'per', 'si', 'nos', 'regañ']
['lueg', 'este', 'com', 'hab', 'trabaj', 'que', 'entonc', 'trabaj', 'el', 'hombr']
['no', 'vim', 'ningun', 'de', 'esas', 'que', 'pic', 'fin', 'porqu', 'estab', 'sec']


## 2. Insertar símbolos de inicio y final de cadena.

In [6]:
# Definimos los simbolos
BOS = '<BOS>'
EOS = '<EOS>'
UNK = '<UNK>'

In [7]:
# Definamos una función para sustituir los hapax, esta funcion recibe un corpus
def replace_hapax(corpus):
    # Obtenemos las frecuencias de las palabras del corpus
    freqs = Counter( chain(*[' '.join(sent).split() for sent in corpus]) )

    # Sustituimos los hápax por UNK en las palabras con frecuencia = 1
    corpus_unk = []

    for line in corpus:
        new_line = []
        for word in line:
            # Si la frecuencia de la palabra = 1, es un hapáx
            if freqs[word] == 1:
                new_line.append(UNK)
            # Si no, añadimos la palabra original
            else:
                new_line.append(word)
        corpus_unk.append(new_line)
    return corpus_unk

In [8]:
# Sustituimos los hápax en los corpus

train_corpus_unk = []
train_corpus_unk = replace_hapax(train_corpus)
print("### Corpus de entrenamiento ###")
print("\nTamaño del corpus de entrenamiento:",len(train_corpus_unk), " lineas")
ejemplos("\nLineas de ejemplo del corpus de entrenamiento", train_corpus_unk, 5)

eval_corpus_unk = []
eval_corpus_unk = replace_hapax(eval_corpus)
print("\n### Corpus de evaluacion ###")
print("\nTamaño del corpus de evaluación:", len(eval_corpus_unk), " lineas")
ejemplos("\nLineas de ejemplo del corpus de evaluacion", eval_corpus_unk, 5)

### Corpus de entrenamiento ###

Tamaño del corpus de entrenamiento: 751  lineas

Lineas de ejemplo del corpus de entrenamiento
['pues', 'lo', 'pag', 'y', 'estuv', 'bien', 'entonc']
['voy', 'a', 'cuid', 'mi', 'poll']
['mis', 'hoj', 'de', 'maiz', 'todav', 'no', 'me', 'las', 'acab', 'nomas', 'estan']
['<UNK>', 'dond', 'ha', 'estad', 'nuestr', 'madr', 'por', 'mas', 'de', 'eso']
['se', 'las', 'llev', 'con', 'burrit', 'nomas']

### Corpus de evaluacion ###

Tamaño del corpus de evaluación: 323  lineas

Lineas de ejemplo del corpus de evaluacion
['per', 'el', 'niñ', 'se', '<UNK>']
['me', 'iba', 'a', '<UNK>', 'por', 'alli', 'hast', 'la', 'tard']
['pues', 'si', 'pues', 'no', 'per', 'si', 'nos', 'regañ']
['lueg', 'este', 'com', 'hab', 'trabaj', 'que', 'entonc', 'trabaj', 'el', 'hombr']
['no', '<UNK>', '<UNK>', 'de', 'esas', 'que', 'pic', '<UNK>', 'porqu', 'estab', '<UNK>']


In [9]:
# Definamos una función para insertar los BOS y EOS
def insert_simbols_of_sentence(corpus):
    corpus_w_simbols = [ [BOS] + line + [EOS] for line in corpus ]
    return corpus_w_simbols

In [10]:
# Insertamos BOS y EOS
train_corpus_w_simbols = []
train_corpus_w_simbols = insert_simbols_of_sentence(train_corpus_unk)
print("### Corpus de entrenamiento ###")
print("\nTamaño del corpus de entrenamiento:",len(train_corpus_w_simbols), " lineas")
ejemplos("\nLineas de ejemplo del corpus de entrenamiento", train_corpus_w_simbols, 5)

eval_corpus_w_simbols = []
eval_corpus_w_simbols = insert_simbols_of_sentence(eval_corpus_unk)
print("\n### Corpus de evaluacion ###")
print("\nTamaño del corpus de evaluacion:",len(eval_corpus_w_simbols), " lineas")
ejemplos("\nLineas de ejemplo del corpus de evaluacion", eval_corpus_w_simbols, 5)

### Corpus de entrenamiento ###

Tamaño del corpus de entrenamiento: 751  lineas

Lineas de ejemplo del corpus de entrenamiento
['<BOS>', 'pues', 'lo', 'pag', 'y', 'estuv', 'bien', 'entonc', '<EOS>']
['<BOS>', 'voy', 'a', 'cuid', 'mi', 'poll', '<EOS>']
['<BOS>', 'mis', 'hoj', 'de', 'maiz', 'todav', 'no', 'me', 'las', 'acab', 'nomas', 'estan', '<EOS>']
['<BOS>', '<UNK>', 'dond', 'ha', 'estad', 'nuestr', 'madr', 'por', 'mas', 'de', 'eso', '<EOS>']
['<BOS>', 'se', 'las', 'llev', 'con', 'burrit', 'nomas', '<EOS>']

### Corpus de evaluacion ###

Tamaño del corpus de evaluacion: 323  lineas

Lineas de ejemplo del corpus de evaluacion
['<BOS>', 'per', 'el', 'niñ', 'se', '<UNK>', '<EOS>']
['<BOS>', 'me', 'iba', 'a', '<UNK>', 'por', 'alli', 'hast', 'la', 'tard', '<EOS>']
['<BOS>', 'pues', 'si', 'pues', 'no', 'per', 'si', 'nos', 'regañ', '<EOS>']
['<BOS>', 'lueg', 'este', 'com', 'hab', 'trabaj', 'que', 'entonc', 'trabaj', 'el', 'hombr', '<EOS>']
['<BOS>', 'no', '<UNK>', '<UNK>', 'de', 'esas', 'q

## 3. Obtener los bigramas que aparecen en el texto (indexar numéricamente).

In [20]:
# Definamos una función para indexar numericamente
def corpus2index(corpus):
    # Creamos un diccionario para los stems
    vocab = defaultdict()
    vocab.default_factory = lambda: len(vocab)

    # Creamos el corpus con sus indices
    corpus_ids = [[vocab[word] for word in line] for line in corpus]
    return corpus_ids, vocab

In [21]:
# Indexamos numericamente los stems

train_corpus_ids, train_vocab = corpus2index(train_corpus_w_simbols)
print("### Corpus de entrenamiento ###")
ejemplos("\nOraciones en corpus de entrenamiento", train_corpus_w_simbols, 5)
ejemplos("\nIndices en corpus de entrenamiento", train_corpus_ids, 5)

eval_corpus_ids, eval_vocab = corpus2index(eval_corpus_w_simbols)
print("\n### Corpus de evaluacion ###")
ejemplos("\nOraciones en corpus de evaluacion", eval_corpus_w_simbols, 5)
ejemplos("\nIndices en corpus de evaluacion", eval_corpus_ids, 5)

### Corpus de entrenamiento ###

Oraciones en corpus de entrenamiento
['<BOS>', 'pues', 'lo', 'pag', 'y', 'estuv', 'bien', 'entonc', '<EOS>']
['<BOS>', 'voy', 'a', 'cuid', 'mi', 'poll', '<EOS>']
['<BOS>', 'mis', 'hoj', 'de', 'maiz', 'todav', 'no', 'me', 'las', 'acab', 'nomas', 'estan', '<EOS>']
['<BOS>', '<UNK>', 'dond', 'ha', 'estad', 'nuestr', 'madr', 'por', 'mas', 'de', 'eso', '<EOS>']
['<BOS>', 'se', 'las', 'llev', 'con', 'burrit', 'nomas', '<EOS>']

Indices en corpus de entrenamiento
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[0, 9, 10, 11, 12, 13, 8]
[0, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 8]
[0, 25, 26, 27, 28, 29, 30, 31, 32, 16, 33, 8]
[0, 34, 21, 35, 36, 37, 23, 8]

### Corpus de evaluacion ###

Oraciones en corpus de evaluacion
['<BOS>', 'per', 'el', 'niñ', 'se', '<UNK>', '<EOS>']
['<BOS>', 'me', 'iba', 'a', '<UNK>', 'por', 'alli', 'hast', 'la', 'tard', '<EOS>']
['<BOS>', 'pues', 'si', 'pues', 'no', 'per', 'si', 'nos', 'regañ', '<EOS>']
['<BOS>', 'lueg', 'este', 'com', 'hab', 'trab

In [13]:
# Definamos una función para crear los bigrama
def corpus2bigrams(corpus):
    bigrams = list(chain(*[zip(cad,cad[1:]) for cad in corpus]))
    return bigrams

In [14]:
# Obtencion de los bigramas
train_bigrams = []
train_bigrams = corpus2bigrams(train_corpus_ids)
print("### Corpus de entrenamiento ###")
print("\nNúmero de bigramas:", len(train_bigrams))
print("\nEjemplos de bigramas de entrenamiento\n", train_bigrams[:50])

eval_bigrams = []
eval_bigrams = corpus2bigrams(eval_corpus_ids)
print("\n### Corpus de evaluacion ###")
print("\nNúmero de bigramas:", len(eval_bigrams))
print("\nEjemplos de bigramas de evaluacion\n", train_bigrams[:50])

### Corpus de entrenamiento ###

Número de bigramas: 6966

Ejemplos de bigramas de entrenamiento
 [(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (0, 9), (9, 10), (10, 11), (11, 12), (12, 13), (13, 8), (0, 14), (14, 15), (15, 16), (16, 17), (17, 18), (18, 19), (19, 20), (20, 21), (21, 22), (22, 23), (23, 24), (24, 8), (0, 25), (25, 26), (26, 27), (27, 28), (28, 29), (29, 30), (30, 31), (31, 32), (32, 16), (16, 33), (33, 8), (0, 34), (34, 21), (21, 35), (35, 36), (36, 37), (37, 23), (23, 8), (0, 38), (38, 39), (39, 1), (1, 34), (34, 40), (40, 8)]

### Corpus de evaluacion ###

Número de bigramas: 2906

Ejemplos de bigramas de evaluacion
 [(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (0, 9), (9, 10), (10, 11), (11, 12), (12, 13), (13, 8), (0, 14), (14, 15), (15, 16), (16, 17), (17, 18), (18, 19), (19, 20), (20, 21), (21, 22), (22, 23), (23, 24), (24, 8), (0, 25), (25, 26), (26, 27), (27, 28), (28, 29), (29, 30), (30, 31), (31, 32), (32, 16), (16, 33), 

## 4. Entrenar con los bigramas la red neuronal y obtener los valores para los hiperparámetros. Tomar de 100 unidades para la primera capa oculta (capa lineal) y 300 para la segunda capa oculta (capa con tanh).`

In [18]:
# Definimos hiperparametros de la red
T = 100    # Numero de iteraciones
d = 100    # Dimension de los embeddings
m = 300    # Numero de unidades en la capa oculta
n = 0.1    # Rango de aprendizaje
N = len(train_vocab) # Tamaño del vocabulario

## 5. Obtener las matrices A y Π a partir de las salidas de la red neuronal (probabilidad Softmax).

In [15]:
# Modelo de lenguaje μ = (Σ,A,Π), 
# Σ es un conjunto de símbolos o vocabulario
# A = (ai j) = p(wj|wi) son las probabilidades de transicion de los bigramas
# Π = (πi) = p(wi) son las probabilidades iniciales.
# Para determinar la probabilidad de cualquier cadena en un lenguaje
# cuyo vocabulario sea Σ se requiere unicamente conocer N2 
# probabilidades de transicion y N probabilidades iniciales, donde
# |Σ| = N es el tamano del vocabulario.

## 6. Evaluar el modelo (con Entropía).

## 7. Calcular la probabilidad de las siguientes oraciones:
   - Nos bañamos con agua caliente
   - El animalito le olía la cabeza
   - Pascuala ordeñaba las vacas